## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-18-18-56-39 +0000,nypost,Investigators probing ties between Brown Unive...,https://nypost.com/2025/12/18/us-news/investig...
1,2025-12-18-18-54-07 +0000,bbc,Kennedy Center to be renamed Trump-Kennedy Cen...,https://www.bbc.com/news/articles/cp84pxvp87eo...
2,2025-12-18-18-52-05 +0000,startribune,Prosecutor says 14 Minnesota programs are targ...,https://www.startribune.com/prosecutors-charge...
3,2025-12-18-18-52-00 +0000,wsj,Kennedy Center Board Votes to Add Trump to Ven...,https://www.wsj.com/us-news/kennedy-center-boa...
4,2025-12-18-18-51-09 +0000,nyt,Trump to Sign Executive Order to Reclassify Ma...,https://www.nytimes.com/2025/12/18/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2348/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,66
181,new,22
17,house,15
211,bondi,15
54,billion,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
339,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...,166
372,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...,149
13,2025-12-18-18-38-09 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...,117
233,2025-12-18-06-03-00 +0000,wsj,President Trump defended his handling of the e...,https://www.wsj.com/politics/policy/trump-defe...,115
69,2025-12-18-17-02-00 +0000,wsj,Trump Media to Merge With Nuclear Fusion Firm ...,https://www.wsj.com/business/deals/trump-media...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
339,166,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...
372,76,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...
376,60,2025-12-17-19-55-55 +0000,nypost,Top cop defends Bondi Beach officer accused of...,https://nypost.com/2025/12/17/world-news/top-c...
117,56,2025-12-18-14-55-00 +0000,wsj,The Bank of England cut its key interest rate ...,https://www.wsj.com/economy/central-banking/ba...
362,53,2025-12-17-20-36-00 +0000,wsj,FCC Chair Brendan Carr told a Senate oversight...,https://www.wsj.com/politics/policy/fcc-isnt-a...
195,53,2025-12-18-11-00-00 +0000,wsj,A Billion-Dollar Question Hangs Over the New A...,https://www.wsj.com/articles/a-billion-dollar-...
114,46,2025-12-18-15-03-00 +0000,wsj,Oil Extends Gains With Market Watching Venezue...,https://www.wsj.com/finance/commodities-future...
218,45,2025-12-18-09-00-00 +0000,cbc,Trump hasn't threatened ripping up North Ameri...,https://www.cbc.ca/news/politics/mark-carney-d...
311,41,2025-12-17-23-06-12 +0000,nypost,Senate approves bill inspired by DC plane cras...,https://nypost.com/2025/12/17/us-news/senate-a...
318,40,2025-12-17-22-45-06 +0000,bbc,Rob Reiner's family breaks silence as son Nick...,https://www.bbc.com/news/articles/cx239pp7gl7o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
